PRAC01 Tipologia UOC Curso 2022

Scrapper de WEB http://www.emagister.com

Equipo: Angel A. Urbina & Almudena Caballero

Fecha: 02 Abril 2022

Versión: 1.1

Notas: Incluye análisis Secuencial y Multiproceso


In [25]:
# Construcción de Loggings para informar

# Objetivo: Disponer tanto en pantalla como en file (log) de el procesamiento que se realiza 

import logging
import logging.config

# https://coderzcolumn.com/tutorials/python/logging-config-simple-guide-to-configure-loggers-from-dictionary-and-config-files-in-python

# Carga de Archivo de configuracón del login
logging.config.fileConfig('loggingUOCPRA01.conf')

# Creacción del logger
logger = logging.getLogger('UOCPRA01')

# 'application' code
#logger.debug('debug message')
#logger.info('info message')
#logger.warning('warn message')
#logger.error('error message')
#logger.critical('critical message')

In [16]:
# Librerias Necesarias

from urllib.request import urlopen
from urllib.error import HTTPError
from urllib.error import URLError
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np
import re

import os

import requests
import random
import time
from time import sleep

# Normalización textos (quitar codigos con acentos)
from unicodedata import normalize


# 00.- Funciones Auxiliares

In [3]:
# Extraccion informacion del nombre archivo de un URL
# Función Auxiliar

# https://stackoverflow.com/questions/66876071/extracting-a-complex-substring-using-regex-with-data-from-a-string-in-python
# https://stackoverflow.com/questions/14473180/regex-to-get-a-filename-from-a-url

def ExtraeName(cadena):
    """
    Función extraccion informacion del nombre archivo de un URL
    ExtraeName(cadena)
    entrada:
        cadena --> Url
    return:
        string final del url
    """
    
    regex=r"(?<=\/)[^\/\?#]+(?=[^\/]*$)"
    matches = re.findall(regex, cadena)
    return str(matches)


In [4]:
# Generación Directorio y Devlución Path Archivo al Directorio
# Funcion Auxiliar 

def GenDirecFile(path, filename):
    """
    GenDirecFile(path, filename, r)
    Entrada:
        path -> Path del directorio a generar
        filename -> Nombre Archivo a guardar
    Devuelve:
        path y filename
    """
    # Generación directorio si no existe
    if not os.path.exists(path):
        os.makedirs(path)
    # Grabación en el directorio en formato binario (imagenes)
    return os.path.join(path, filename)

In [30]:
# Descarga Imagenes
# Funcion Auxiliar 

def DescargaImagenes(path, link, institucion):
    """
    DescargaImagenes(path, link, institucion)
    Entrada:
        path -> Path del directorio a generar
        link -> Link a el archivo a descargar
        institucion -> Nombre de la institucion cuyo logo es la imagen
    Devuelve:
        Genera Archivo en directorio path
    """
    # Delays para evitar problemas
    time.sleep(random.randint(1,2))
    # Obtener Imagen
    #r = requests.get(s, allow_redirects=False)   
    r = requests.get(link, allow_redirects=True)
    # Generar Nombre Archivo
    nombre = str(institucion)
    # Directorio donde se generara salida
    file01=GenDirecFile(str(path), nombre)
    # Grabacion resultados
    with open(file01, 'wb') as file:
        file.write(r.content)

In [6]:
# Grabación Archivo en Directorio
# Funcion Auxiliar 

# path = IMAGENES
# filename = img_alt + '.jpg'
# buffer = r.content
# open(nombre, 'wb').write(r.content)

def GraFileDirec(path, filename, r):
    """
    GraFileDirec(path, filename, r)
    path -> Path del directorio a generar
    filename -> Nombre Archivo a guardar
    r --> Objeto BeatifulSoup
        r.content -> Contenido Stream del Objeto
    """
    # Generación directorio si no existe
    if not os.path.exists(path):
        os.makedirs(path)
    # Grabación en el directorio en formato binario (imagenes)
    with open(os.path.join(path, filename), 'wb') as temp_file:
        temp_file.write(r.content)

# 01.- Análisis de Links asociados a la WEB

In [ ]:
# 01.01- El primer paso es la identificación de los links asociados a cada Master

# Pagina WEB a analizar

URLWebPageAnalizar = 'https://www.emagister.com/master/web/sitemap'

logger.info('Analisis WEB %s', URLWebPageAnalizar)

# Lista donde se almacenaran los links encontrados
ListaWebs=[]

# Abrir Web
html = urlopen(URLWebPageAnalizar)

# Generar objeto BeatifulSoup
bs = BeautifulSoup(html, 'html.parser')
for link in bs.find_all('a'):
    # Busca todos los elementos que esten en 'href'
    try:
        StringWeb = link.attrs['href']
    except:
        continue
    # Comprueba que lo obtenido se corresponda con el formato de una pagina WEB
    # match (True si es un link / False en resto)
    match = re.search(r'https?://[^\s<>"]+|www\.[^\s<>"]+', str(StringWeb)) 
    if match:      
        ListaWebs.append(link.attrs['href'])
        logger.info('Encontrado link %s', link.attrs['href'])

# Guardar links obtenidos en Archivo Auxiliar FILE_Links
# - Nombre Archivo
FILE_Links = '01_ListaWebs.xlsx'
# - Genera DataFrame apartir de la lista de WEBs generadas
df = pd.DataFrame(ListaWebs)
# - Escribe Archivo
df.to_excel(FILE_Links)
logger.info('Generado Archivo %s', FILE_Links)

In [ ]:
# Nos informa del Número Total de Links encontrados
logger.info('Encontrado %s links', len(ListaWebs))

In [ ]:
# 01.02.- Función para Construcción del Diccionario de Información de los links encontrados.

diccionario={}

def RecorrerWebs(ListaWebs):
    """
    Construye Diccionario Valores de una lista de Webs
    
    Funcion:
       RecorrerWebs( ListaWeb)
    Entrada:
        ListaWeb --> Lista de Direcciones de Web
    Salida:
        Diccionario con información:
            Keys: 
                Indice correlativo
            Values:
                - Pagina donde buscamos
                - Titulo Master
                - Link al master concreto
    """
    
    # Leer Webs obtenidas paso previo
    # - indice
    indice = 0
    # - Delay 1 seg para no ser banneado
    sleep(1)
    
    for indi, ListaWeb in enumerate(ListaWebs):
        # Delay 1 seg para no ser banneado
        sleep(1)
        # Captura de posibles errores Webs
        try:
            html = urlopen(ListaWeb)
        except HTTPError as e:
            # Lanza un aviso con indice de ListaWebs y error
            logger.warning('%s warning %s',indi, e)
            continue
        except URLError as e:
            # Lanza aviso si no encuentra la Web
            logger.warning('%s ',ListaWeb)
            logger.warning('%s ',e)               
            continue
        # Captura posibles errores Beatifulsoup
        try:
            bs = BeautifulSoup(html, 'html.parser')
        except AttributeError as e:
            logger.warning('%s warning %s',indi, e)              
            continue
        
        # Captura de información básica
        divs = bs.find_all(class_ ="course-box-item title-box")
    
        for s in divs:
            inks = s.find_all('h3')
            for t in inks:
                lista = t.find_all('a')
                # Obtención indice y valor de lista
                #for indice, x in enumerate(lista):
                for x in lista:
                    indice = indice+1
                    # Titulo 
                    x2 = x.attrs['title']
                    # Referencia al titulo
                    x1 = x.attrs['href']
                    logger.info('%s: %s',indice, x2)

                    # Construccion diccionario salida
                    diccionario[indice] = [ListaWeb,x2,x1]
    return diccionario

In [ ]:
# 01.03.- Construcción Efectiva del Diccionario

# Nombre Diccionario Construido:
#    a --> diccionario

a = RecorrerWebs(ListaWebs)

logger.info('Construido Diccionario')

01.04.- Guardamos la información obtenida en un Archivo para ello:

        01.04.01.- Generamos Dataframe
        01.04.02.- Generamos File a partir del Dataframe

 Pasos:

In [ ]:
# 01.04.01.- Generación de Dataframe a partir del Diccionario

# https://stackoverflow.com/questions/54416620/dictionary-with-multiple-key-values-to-dataframe

TablaResult = (pd.DataFrame.from_dict(a, orient='index')
                  .rename(columns=lambda x: x+1)
                  .add_prefix('n')
                  .rename_axis('indice')
                  .reset_index())

logger.info('Construida Tabla TablaResult')

# Poner nombres columnas
TablaResult.columns = ['key','Pagina_Origen', 'Titulo', 'Pag_Referencia']

# Poner de indice a key
TablaResult=TablaResult.set_index('key')

# Verificación Construcción Ok( No necesario )
# TablaResult.head(3)

logger.info('Tamaño tabla  %s', TablaResult.shape)
logger.info('Columnas  %s', TablaResult.columns)

In [ ]:
# 01.04.02.- Generación Archivo parcial de resultados

# Generación Nombres Archivos a generar
from datetime import datetime

nombre=datetime.today().strftime('%d-%m-%y')

FILE_01 = "LINK-"+nombre+".xlsx"

TablaResult.to_excel(FILE_01)
logger.info('Generado Archivo %s', FILE_01)

# 02.- Analisis de Links Identificados 

    02.1- Para ello leeremos Archivo del paso previo
    02.2- Haremos el analisis

In [7]:
# 02.01.- Identificación Archivos en nuestra Maquina

# Directorio donde estamos
cwd=os.getcwd()
print("-----------------------------------------------------------------------")
print("A.- Directorio en maquina donde estamos")
print (cwd)

# Lista de Archivos disponibles en directorio 'Directorio'
Directorio = cwd
print("-----------------------------------------------------------------------")
print("C.- Listado de Archivos Disponibles")
ListaFile=os.listdir(Directorio)

# Selección Archivos que hayan sido generados en lanzamientos previos del pto 01.
# - Uso Regex 
cadena=r"^LINK-"
prog1 = re.compile(cadena)

# Extraer lista de archivos que cumplan criterio

Archivos_FILT=[]
for s in ListaFile:
    if prog1.search(s):
        Archivos_FILT.append(s)
# Listado de Nombres Archivos validos
Archivos_FILT

-----------------------------------------------------------------------
A.- Directorio en maquina donde estamos
/home/anson/Descargas/01_UOC/05_TIPOLOGIA/PRA01
-----------------------------------------------------------------------
C.- Listado de Archivos Disponibles


['LINK-26-03-22.xlsx', 'LINK-27-03-22.xlsx', 'LINK-02-04-22.xlsx']

In [8]:
# Escogeremos el archivo que nos interese de los identificados en el paso previo

# Leer Archivo
FILE_02 = 'LINK-27-03-22.xlsx'

df = pd.read_excel(FILE_02, index_col=None)

logger.info('Leido Archivo %s', FILE_02)

2022-04-02 19:12:46,557 - UOCPRA01 - INFO - Leido Archivo LINK-27-03-22.xlsx


In [9]:
# Función Lectura Información Webs secundarias de Masters

def WebSecundaria(Link):
    """
    Funcion:
        WebSecundaria(link, indice)
        Version 2.0
    Entrada:
        link -> Link a analizar
     
    Return:
        Lista valores obtenidos
        - Titulo -> Titulo Master
        - Entidad -> Nombre entidad que hace master
        - urlImagen -> Link Imagen
        - nombreImagen -> Nombre Imagen
        - Telefono -> Telefono contacto
        - Precio -> Precio Master
        - Metodologia -> Metodologia Master
        - Lugar -> Lugar Imparticion
        - Duracion -> Duracion Master
        - Tipologia -> Tipologia Master
        - BolsaEmp -> Bolsa Empleo
        - x3 -> Precio Master
        - x4 -> Texto Descriptivo Master
        - x5 -> Tipologia Master
        - x6 -> Programa Master
    """
    # Abrir Web    
    # Captura de posibles errores Webs
    
    # Delays para evitar problemas
    time.sleep(random.randint(1,2))
    
    try:
        html = urlopen(Link)
    except HTTPError as e:
        # Lanza un aviso con indice de ListaWebs y error
        logger.warning('warning %s', e)
        return
    except URLError as e:
        # Lanza aviso si no encuentra la Web
        logger.warning('%s ',e)               
        return

    # Generar objeto BeatifulSoup
    bs = BeautifulSoup(html, 'html.parser')
                       
    # Titulo Master
    for t1 in bs.select(".title-box__name"):
        #print(t1.text)
        x1 = t1.text
        Titulo = x1.strip()
                       
    # Nombre entidad que hace el Master
    for t1 in bs.select(".course-box__link"):
        #print(t1.text)
        Entidad = t1.text.strip()
        
    # Imagen en pagina Master
    a=bs.select(".boxes-untrack__logo > img:nth-child(1)")
    # Si hay imagen
    if a:
        # Link Imagen
        urlImagen=a[0]['src'].strip()
        # Extraer nombre Archivo Imagen
        nombreImagen= ExtraeName(urlImagen)
    else:
        urlImagen=""
        nombreImagen="Sin Imagen"

    # Telefono
    a=bs.select(".app_contactCenterSectionDesktop:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > a:nth-child(1) > span:nth-child(1)")
    if a:
        # Extracción Telefono
        Telefono=a[0].text
        # - Elimina caracteres especiales
        Telefono=re.sub(r"[^a-zA-Z0-9]","",Telefono)
        # - Elimina espacios blanco principio y final
        Telefono=Telefono.strip()
    else:
        Telefono="Sin Datos"
        
     # Precio
    a=bs.select(".app_course_price_box > div:nth-child(2) > div:nth-child(1) > span:nth-child(1)")
    if a:
        # Extracción Precio
        Precio=a[0].text
        # - Elimina caracteres especiales
        Precio=re.sub(r"[^a-zA-Z0-9]","",Precio)
        # - Elimina espacios blanco principio y final
        Precio=Precio.strip()
        # - Si todos los caracteres son numericoa convierte a float
        if Precio.isnumeric():
            # Convertir en float
            Precio=float(Precio)
        
    else:
        Precio="Precio a consultar"
    
    # Metodologia
    a=bs.select("ul.course-detail__list:nth-child(1) > li:nth-child(2) > p:nth-child(2) > span:nth-child(1)")
    if a:
        # Metodologia
        Metodologia=a[0].text.strip()
    else:
        Metodologia="Sin Datos"
    
    # Lugar impartición Master
    a=bs.select("ul.course-detail__list:nth-child(1) > li:nth-child(3) > p:nth-child(2) > span:nth-child(1)")
    if a:
        # Lugar
        Lugar=a[0].text.strip()
    else:
        Lugar="Sin Datos"
    
    # TipoCurso
    a=bs.select("ul.course-detail__list:nth-child(1) > li:nth-child(4) > p:nth-child(2) > span:nth-child(1)")
    if a:
        # TipoCurso
        TipoCurso=a[0].text.strip()
    else:
        TipoCurso="Sin Datos"
       
    # Tipologia
    a=bs.select("ul.course-detail__list:nth-child(1) > li:nth-child(1) > p:nth-child(2) > span:nth-child(1)")
    if a:
        # Tipologia
        Tipologia=a[0].text.strip()
    else:
        Tipologia="Sin Datos" 

     # Bolsa Empleo
    a=bs.select("ul.course-detail__list:nth-child(2) > li:nth-child(4) > p:nth-child(2) > span:nth-child(1)")
    if a:
        # Bolsa Empleo (Si/No)
        BolsaEmpl=a[0].text.strip()
    else:
        BolsaEmpl="Sin Datos" 
                       
    # Precio del Master
    # - Si no tiene valor asigna precio Sin Datos
    if not bs.select("div.price-box:nth-child(1) > div:nth-child(1) > div:nth-child(1) > span:nth-child(1)"):
        x3= "SinDatos"
    # - En el resto de casos
    for t1 in bs.select("div.price-box:nth-child(1) > div:nth-child(1) > div:nth-child(1) > span:nth-child(1)"):
        # - Obtención del valor
        x31 = t1.text
        # - Elimina caracteres especiales
        x3=re.sub(r"[^a-zA-Z0-9]","",x31)
        # - Elimina espacios blanco principio y final
        x3=x3.strip()
        # - Si todos los caracteres son numericoa convierte a float
        if x3.isnumeric():
            # Convertir en float
            x3=float(x3)
            # Definirlo como precio ( La información esta en dos sitios )
            Precio=x3
        
    # Texto Descriptivo
    # - Si no tiene valor asigna cadena vacia
    if not bs.select(".course-box__text"):
        x4="SinDatos"
    # - En el resto de casos
    for t1 in bs.select(".course-box__text"):
        # - Obtención del valor
        x41 = t1.text
        # - Eliminamos acentos. Basado en:
# https://es.stackoverflow.com/questions/135707/c%C3%B3mo-puedo-reemplazar-las-letras-con-tildes-por-las-mismas-sin-tilde-pero-no-l
        # -> NFD y eliminar diacríticos
        x4 = re.sub(r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", 
        normalize("NFD", x41), 0, re.I)
        # -> NFC
        x42 = normalize( 'NFC', x4)
        
        # - Eliminamos caracteres especiales
        x4=re.sub(r"[^a-zA-Z0-9]"," ",x42)
        # - Elimina espacios blanco principio y final
        x4=x4.strip()
        
        
    # Tipologia del Master
    # - Si no tiene valor asigna cadena vacia
    if not bs.select(".course-venues__address > span:nth-child(1)"):
        x5=""
    # - En el resto de los casos
    for t1 in bs.select(".course-venues__address > span:nth-child(1)"):
        # Si no hay valores
        if not t1:
            x5="Sin Datos"
        #print(t1.text)
        x5 = t1.text.strip()

    # Programa Master
    texto=[]
    for table in bs.select(".lessons-box.app_lessons-box"):
        for i, tr in enumerate(table.findAll('strong')):
            texto.append(tr.text)
            for td in tr.findAll('li'):
                texto.append(td.text)
    x6 = texto
                       
    # Construcción Lista resultado 
#        - Titulo -> Titulo Master -- 'MASTER'                   
#        - Entidad -> Nombre entidad que hace master -- 'ENTIDAD'
#        - Precio -> Precio Master -- 'PRECIO'
#        - TipoCurso -> Tipo Curso -- 'TIPO_CURSO'
#        - Tipologia -> Tipologia Master -- 'TIPOLOGIA'
#        - Metodologia -> Metodologia Master -- 'METODOLOGIA'
#        - Lugar -> Lugar Imparticion -- LUGAR
#        - BolsaEmp -> Bolsa Empleo -- 'BOLSAEMP'
#        - Telefono -> Telefono contacto --  'TELEFONO'
#        - x4 -> Texto Descriptivo Master
#        - x6 -> Programa Master  
#        - urlImagen -> Link Imagen
#        - nombreImagen -> Nombre Imagen
#        'WEB'

#        - x3 -> Precio Master (NO LO USAREMOS)
#        - x5 -> Tipologia Master (NO LO USAREMOS)
  
    
    logger.info('Procesada %s', x1)                  
    return (Titulo,
            Entidad,
            Precio,
            Tipologia,
            Metodologia,           
            Lugar,          
            BolsaEmpl,
            Telefono,
            x4,
            x6,
            urlImagen,
            nombreImagen,           
            Link)

# 03.- ANALISIS

Tenemos dos opciones:
    
    03.01- SECUENCIAL
    03.02.-MULTIPROCESO

In [ ]:
# 03.01.-PROCESO SECUENCIAL de generación Datos Master

# Datos --> DataFrame df
#  - Columnas:
#       'key'-> Indice
#       'Pagina_Origen' -> Web Origen Busqueda
#     , 'Titulo' -> Nombre Master
#     , 'Pag_Referencia' --> Web de detalle Master
#     

# Construcción Lista Webs a analizar
listadoWebs = df['Pag_Referencia'].tolist()
# Tamaño Datos
#tamano = len(listadoWebs)
tamano = 5

# Control Tiempo
start = time.time()

logger.info('********************************')
logger.info('*   INICIO PROCESO SECUENCIAL  *')
logger.info('********************************')

# Para cada valor del indice
for indice in range(tamano):
    # Paro para evitar bloqueo 
    sleep(1)
    # Construcción Diccionario
    #s = WebSecundaria(listadoWebs[indice], indice)
    resultado_seq = WebSecundaria(listadoWebs[indice])
    #logger.info('%s: Informacion de %s', indice, s[indice][2])

logger.info('********************************')
logger.info('*     FIN PROCESO SECUENCIAL   *')
logger.info('********************************')

end = time.time()
duracion = (end-start)
logger.info('El proceso secuencial duro %s segundos', duracion)

In [10]:
# 03.02.-PROCESO USANDO MULTIPROCESO de generación Datos Master

from multiprocessing import Pool

In [11]:
# Lista de web a analizar

# Construcción Lista Webs a analizar
listadoWebs = df['Pag_Referencia'].tolist()


# Para hacer pruebas
url_list=listadoWebs[0:1000]

In [12]:
url_list

['https://www.emagister.com/master-mba-direccion-centros-hidrotermales-cursos-3572081.htm',
 'https://www.emagister.com/master-biotecnologia-salud-madrid-cursos-3170196.htm',
 'https://www.emagister.com/master-universitario-nutricion-salud-cursos-3006594.htm',
 'https://www.emagister.com/posgrado-nutricion-salud-distancia-cursos-3693696.htm',
 'https://www.emagister.com/gestion-sanitaria-para-profesionales-salud-cursos-2801676.htm',
 'https://www.emagister.com/master-salud-mental-cursos-3466528.htm',
 'https://www.emagister.com/master-trabajo-social-servicios-salud-master-salud-mental-cursos-3658125.htm',
 'https://www.emagister.com/master-salud-mental-anciano-master-enfermedad-del-alzheimer-practicas-cursos-3544150.htm',
 'https://www.emagister.com/master-direccion-centros-medicos-clinicas-hospitales-mas-master-gerencia-salud-cursos-3202586.htm',
 'https://www.emagister.com/posgrado-nutricion-salud-cursos-3693695.htm',
 'https://www.emagister.com/master-online-humanizacion-salud-curso

In [17]:
# https://medium.com/@kunal.rustagi/boost-your-web-crawler-using-multiple-processes-in-python-3cc3ff519226

start = time.time()

logger.info('********************************')
logger.info('*   INICIO MULTIPROCESO        *')
logger.info('********************************')

p = Pool(10)
resultados_mul = p.map(WebSecundaria, url_list)
p.terminate()
p.join()



logger.info('********************************')
logger.info('*     FIN MULTIPROCESO         *')
logger.info('********************************')

end = time.time()
duracion = (end-start)
logger.info('La secuencia Multiproceso duro %s segundos', duracion)

2022-04-02 19:37:38,909 - UOCPRA01 - INFO - ********************************
2022-04-02 19:37:38,911 - UOCPRA01 - INFO - *   INICIO MULTIPROCESO        *
2022-04-02 19:37:38,912 - UOCPRA01 - INFO - ********************************
2022-04-02 19:37:41,222 - UOCPRA01 - INFO - Procesada  MÁSTER EN ELABORACIÓN DE FÓRMULAS MAGISTRALES PARA FARMACIA - CON CERTIFICACIÓN UNIVERSITARIA INTERNACIONAL Y RECONOCIMIENTO DE 30 ECTS 
2022-04-02 19:37:41,241 - UOCPRA01 - INFO - Procesada  Experto Universitario en triaje para Enfermería de la SEMES 
2022-04-02 19:37:41,278 - UOCPRA01 - INFO - Procesada  Máster MBA en dirección de centros hidrotermales 
2022-04-02 19:37:42,241 - UOCPRA01 - INFO - Procesada  Master en Gestión Sanitaria 
2022-04-02 19:37:42,376 - UOCPRA01 - INFO - Procesada  Máster en tratamiento de lesiones deportivas 
2022-04-02 19:37:42,422 - UOCPRA01 - INFO - Procesada  Máster en homeopatía, fitoterapia y nutrición 
2022-04-02 19:37:42,436 - UOCPRA01 - INFO - Procesada  Postgrado Expe

2022-04-02 19:37:58,602 - UOCPRA01 - INFO - Procesada  POSTGRADO EN INTEGRACIÓN SOCIAL DEL ANCIANO 
2022-04-02 19:37:58,677 - UOCPRA01 - INFO - Procesada  MÁSTER EN ACUPUNTURA - ONLINE 
2022-04-02 19:37:58,891 - UOCPRA01 - INFO - Procesada  Máster en rehabilitación deportiva 
2022-04-02 19:37:59,317 - UOCPRA01 - INFO - Procesada  Experto en Ortodoncia Funcional, Aparatologia fija estética y alineadores 
2022-04-02 19:37:59,337 - UOCPRA01 - INFO - Procesada  Máster en Medicina Estética (Con Certificación Universitaria Internacional + Reconocimiento 24 ECTS) 
2022-04-02 19:37:59,339 - UOCPRA01 - INFO - Procesada  Máster experto en logopedia, trastornos del lenguaje y del habla 
2022-04-02 19:37:59,477 - UOCPRA01 - INFO - Procesada  Experto Universitario en Técnicas de Enfermería en Atención al Paciente Urgente 
2022-04-02 19:37:59,665 - UOCPRA01 - INFO - Procesada  Postgrado Experimental en Microbiología Industrial 
2022-04-02 19:38:01,435 - UOCPRA01 - INFO - Procesada  Máster de formaci

2022-04-02 19:38:18,052 - UOCPRA01 - INFO - Procesada  MÁSTER EN PATOLOGÍAS TRAUMATOLÓGICAS Y TRATAMIENTO PARA FISIOTERAPEUTAS - CON CERTIFICACIÓN UNIVERSITARIA INTERNACIONAL Y RECONOCIMIENTO DE 30 ECTS 
2022-04-02 19:38:18,172 - UOCPRA01 - INFO - Procesada  Máster en psicología infantil y adolescente más máster en coaching y en inteligencia emocional infantil y juvenil 
2022-04-02 19:38:18,247 - UOCPRA01 - INFO - Procesada  Máster en logopedia infantil y audición 
2022-04-02 19:38:18,600 - UOCPRA01 - INFO - Procesada  MASTER CERÁMICA DENTAL Y CADCAM 
2022-04-02 19:38:18,927 - UOCPRA01 - INFO - Procesada  MÁSTER EN MEDICINA DEPORTIVA 
2022-04-02 19:38:19,302 - UOCPRA01 - INFO - Procesada  POSTGRADO EXPERTO EN TERCERA EDAD 
2022-04-02 19:38:19,675 - UOCPRA01 - INFO - Procesada  MÁSTER EXPERTO EN HOMEOPATÍA + MÁSTER EXPERTO FITOTERAPIA + MÁSTER EXPERTO NUTRICIÓN 
2022-04-02 19:38:19,845 - UOCPRA01 - INFO - Procesada  Técnico en Salud Mental (10 créditos ECTS) 
2022-04-02 19:38:20,227 - U

2022-04-02 19:38:35,985 - UOCPRA01 - INFO - Procesada  MÁSTER ORTODONCIA y CADCAM PARA PROTÉSICOS 
2022-04-02 19:38:36,676 - UOCPRA01 - INFO - Procesada  Master oficial universitario en dirección y gestión sanitaria 
2022-04-02 19:38:36,678 - UOCPRA01 - INFO - Procesada  Máster en Atención Temprana- 
2022-04-02 19:38:36,702 - UOCPRA01 - INFO - Procesada  Master en Primeros Auxilios 
2022-04-02 19:38:37,293 - UOCPRA01 - INFO - Procesada  Máster en Industria Farmacéutica y Parafarmacéutica - Barcelona 
2022-04-02 19:38:37,328 - UOCPRA01 - INFO - Procesada  Máster Experto en Medicina Alternativa 
2022-04-02 19:38:37,337 - UOCPRA01 - INFO - Procesada  Máster universitario en dirección y gestión de enfermería 
2022-04-02 19:38:37,687 - UOCPRA01 - INFO - Procesada  Máster en psicología holística más máster en coaching 
2022-04-02 19:38:37,758 - UOCPRA01 - INFO - Procesada  Master en Nutrición Deportiva 
2022-04-02 19:38:37,874 - UOCPRA01 - INFO - Procesada  Máster de formación permanente en 

2022-04-02 19:38:55,307 - UOCPRA01 - INFO - Procesada  Máster En Auditorías Y Valoraciones Contables Y Financieras + Perito Judicial - con Prácticas Garantizadas 
2022-04-02 19:38:55,497 - UOCPRA01 - INFO - Procesada  Máster en análisis clínicos en laboratorio para veterinaria 
2022-04-02 19:38:55,899 - UOCPRA01 - INFO - Procesada  Postgrado en e-Commerce Omnichannel 
2022-04-02 19:38:55,927 - UOCPRA01 - INFO - Procesada  Perito judicial + Máster en investigación médico-legal en accidentes de tráfico 
2022-04-02 19:38:56,191 - UOCPRA01 - INFO - Procesada  MÁSTER EN PSICOLOGÍA INFANTIL Y ADOLESCENTE + MÁSTER EN COACHING Y EN INTELIGENCIA EMOCIONAL INFANTIL Y JUVENIL 
2022-04-02 19:38:56,215 - UOCPRA01 - INFO - Procesada  Máster en comercio internacional y logística 
2022-04-02 19:38:56,395 - UOCPRA01 - INFO - Procesada  MÁSTER EN FITOTERAPIA - CERTIFICADO POR APENB - ASOCIACIÓN PROFESIONAL ESPAÑOLA DE NATUROPATÍA Y BIOTERAPIA - 
2022-04-02 19:38:57,109 - UOCPRA01 - INFO - Procesada  MÁS

...

2022-04-02 19:40:59,684 - UOCPRA01 - INFO - Procesada  Pyme: Experto en Gestión de Pymes 
2022-04-02 19:40:59,812 - UOCPRA01 - INFO - Procesada  Máster en Ingeniería de la Calidad 
2022-04-02 19:41:00,113 - UOCPRA01 - INFO - Procesada  Programa Superior en Dirección de Marketing y Ventas 
2022-04-02 19:41:00,760 - UOCPRA01 - INFO - Procesada  Máster en Energías Renovables, Ahorro y Eficiencia Energética_ 
2022-04-02 19:41:00,983 - UOCPRA01 - INFO - Procesada  Máster en Industria Alimentaria – Barcelona 
2022-04-02 19:41:01,662 - UOCPRA01 - INFO - Procesada  Doctorado Doctrinal en Publicidad 
2022-04-02 19:41:01,853 - UOCPRA01 - INFO - Procesada  Máster en Marketing y Comunicación Digital (Semipresencial) 
2022-04-02 19:41:01,902 - UOCPRA01 - INFO - Procesada  Máster en Dirección de Marketing y Gestión Comercial 
2022-04-02 19:41:01,925 - UOCPRA01 - INFO - Procesada 
                     Máster en Dirección de Empresas             
2022-04-02 19:41:01,988 - UOCPRA01 - INFO - Procesada  

2022-04-02 19:41:19,901 - UOCPRA01 - INFO - Procesada  MÁSTER EN DISEÑO DE TUBERÍA INDUSTRIAL 
2022-04-02 19:41:20,325 - UOCPRA01 - INFO - Procesada  Máster en Gestión y Desarrollo de Energías Renovables 
2022-04-02 19:41:20,509 - UOCPRA01 - INFO - Procesada  MÁSTER EN PROCESOS, INSTALACIÓN Y MANTENIMIENTO INDUSTRIAL 
2022-04-02 19:41:21,303 - UOCPRA01 - INFO - Procesada  EXPERTO EN DERMOCOSMÉTICA EN FARMACIAS 
2022-04-02 19:41:22,267 - UOCPRA01 - INFO - Procesada  Curso Universitario Técnico en Sistemas de Gestión Medioambiental ISO 14001 
2022-04-02 19:41:22,938 - UOCPRA01 - INFO - Procesada  MÁSTER EN CALIDAD, MEDIOAMBIENTE, PROCESOS Y PRL 
2022-04-02 19:41:24,405 - UOCPRA01 - INFO - Procesada  MECEA | Máster en Cálculo Estructural Avanzado 
2022-04-02 19:41:24,446 - UOCPRA01 - INFO - Procesada  MÁSTER EN BIOTECNOLOGÍA ALIMENTARIA 
2022-04-02 19:41:25,367 - UOCPRA01 - INFO - Procesada  Master en Educación Social + Titulación Universitaria 
2022-04-02 19:41:26,873 - UOCPRA01 - INFO -

2022-04-02 19:42:18,311 - UOCPRA01 - INFO - Procesada 
                     Máster en Organización Industrial             
2022-04-02 19:42:19,454 - UOCPRA01 - INFO - Procesada  Master en producción Lean y calidad 
2022-04-02 19:42:19,543 - UOCPRA01 - INFO - Procesada  Máster en Coaching Familiar - con Prácticas Garantizadas 
2022-04-02 19:42:19,589 - UOCPRA01 - INFO - Procesada  Máster en Jefe de Taller de Carrocería ONLINE 
2022-04-02 19:42:21,501 - UOCPRA01 - INFO - Procesada  MÁSTER EN TRATAMIENTOS TÉRMICOS EN FABRICACIÓN MECÁNICA 
2022-04-02 19:42:22,231 - UOCPRA01 - INFO - Procesada  Perito Automóviles. Curso Superior de Peritación de Automóviles 
2022-04-02 19:42:22,909 - UOCPRA01 - INFO - Procesada  Máster en Ajuste y Patronaje de Calzado y Complementos 
2022-04-02 19:42:23,194 - UOCPRA01 - INFO - Procesada  Máster en agricultura ecológica más máster en cultivos herbáceos 
2022-04-02 19:42:23,298 - UOCPRA01 - INFO - Procesada  MÁSTER EN MONTAJE Y MANTENIMIENTO DE SISTEMAS DE AU

2022-04-02 19:43:15,545 - UOCPRA01 - INFO - Procesada  Diplomado en ingeniería eléctrica 
2022-04-02 19:43:15,630 - UOCPRA01 - INFO - Procesada  MÁSTER EN INTELIGENCIA EMOCIONAL-- 
2022-04-02 19:43:15,961 - UOCPRA01 - INFO - Procesada  MÁSTER EN ORGANIZACIÓN INDUSTRIAL 
2022-04-02 19:43:16,274 - UOCPRA01 - INFO - Procesada  Magíster en Ingeniería Metalúrgica 
2022-04-02 19:43:16,848 - UOCPRA01 - INFO - Procesada  MÁSTER EN DISEÑO DE CALDERERÍA Y ESTRUCTURAS METÁLICAS 
2022-04-02 19:43:16,882 - UOCPRA01 - INFO - Procesada  Máster en educación social 
2022-04-02 19:43:19,044 - UOCPRA01 - INFO - Procesada  Experto universitario en docencia online. 
2022-04-02 19:43:19,244 - UOCPRA01 - INFO - Procesada  Máster en Gestión Medioambiental 
2022-04-02 19:43:19,875 - UOCPRA01 - INFO - Procesada  Maestría en Medio Ambiente y Responsabilidad Social 
2022-04-02 19:43:19,891 - UOCPRA01 - INFO - Procesada  Programa online, máster en electrónica automotriz 2022. 
2022-04-02 19:43:19,952 - UOCPRA01 - 

2022-04-02 19:43:37,123 - UOCPRA01 - INFO - Procesada  POSTGRADO EN CÁLCULOS DE CALDERERÍA Y ESTRUCTURAS METÁLICAS 
2022-04-02 19:43:38,443 - UOCPRA01 - INFO - Procesada  Máster en neurociencia cognitiva y educación 
2022-04-02 19:43:38,455 - UOCPRA01 - INFO - Procesada  Máster en Gobernanza Educativa 
2022-04-02 19:43:38,836 - UOCPRA01 - INFO - Procesada  Máster Universitario en Gestión Integrada de Prevención, Calidad y Medio Ambiente 
2022-04-02 19:43:39,967 - UOCPRA01 - INFO - Procesada  Máster en Asesoramiento Educativo Familiar 
2022-04-02 19:43:40,457 - UOCPRA01 - INFO - Procesada  Máster Online en Flipped Classroom 
2022-04-02 19:43:40,502 - UOCPRA01 - INFO - Procesada  Máster Universitario en Métodos de Enseñanza en Educación Personalizada 
2022-04-02 19:43:42,006 - UOCPRA01 - INFO - Procesada  Máster universitario en orientación y mediación familiar 
2022-04-02 19:43:45,033 - UOCPRA01 - INFO - Procesada  Curso de Mediación. Experto Universitario en Mediación Familiar. 
2022-0

In [18]:
# 03.03.- Creación Dataframe resultado

DfResult = pd.DataFrame(resultados_mul, columns = ['MASTER',
                                                   'ENTIDAD',
                                                   'PRECIO',
                                                   'TIPOLOGIA',
                                                   'METODOLOGIA',
                                                   'DURACION',
                                                   'BOLSAEMP',
                                                   'TELEFONO',
                                                   'X4',
                                                   'X6',
                                                   'URLIMAGEN',
                                                   'NOMBREIMAGEN',
                                                   'WEB'])

In [19]:
DfResult

,MASTER,ENTIDAD,PRECIO,TIPOLOGIA,METODOLOGIA,DURACION,BOLSAEMP,TELEFONO,X4,X6,URLIMAGEN,NOMBREIMAGEN,WEB
0,Máster MBA en dirección de centros hidrotermales,SELECT BUSINESS SCHOOL,1940.0,Master,Semipresencial,Madrid,Sí,916358357,Plantea tus dudas y otros usuarios podran resp...,"[Contenido formativo, PARTE 1. GESTIÓN LABORAL...",https://www.emagister.com/assets/es/logos/cent...,['l.png'],https://www.emagister.com/master-mba-direccion...
1,Máster en Biotecnología de la Salud - Madrid,CESIF Formación,Precio a consultar,Master,Madrid,1840h,Sin Datos,912669065,Plantea tus dudas y otros usuarios podran resp...,[SECTOR INDUSTRIAL EN BIOTECNOLOGIA DE LA SALU...,https://www.emagister.com/assets/es/logos/cent...,['l.jpg'],https://www.emagister.com/master-biotecnologia...
2,Máster Universitario en Nutrición y Salud,Universidad Internacional de Valencia (VIU),Precio a consultar,Master oficial,Online,Sin Datos,Sin Datos,962642879,El principal objetivo del Master Oficial Onlin...,[],https://www.emagister.com/assets/es/logos/cent...,['l.jpg'],https://www.emagister.com/master-universitario...
3,Posgrado en Nutrición y Salud - A distancia,ISEB - Instituto Superior Europeo de Barcelona,800.0,Postgrado,A distancia,6 Meses,Sin Datos,932824831,Plantea tus dudas y otros usuarios podran resp...,[El Posgrado en Nutrición y Salud se constituy...,https://www.emagister.com/assets/es/logos/cent...,['l.jpg'],https://www.emagister.com/posgrado-nutricion-s...
4,Máster Profesional en Gestión Sanitaria,Cef - Centro de Estudios Financieros,5400.0,Master oficial,Online,Sin Datos,Sin Datos,912669065,Si trabajas en el sector sanitario y quieres a...,[Programa \n\n],https://www.emagister.com/assets/es/logos/cent...,['l.jpg'],https://www.emagister.com/gestion-sanitaria-pa...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Máster en Asesoramiento Educativo Familiar,Universidad Villanueva,3500.0,Master,Sin Datos,Sin Datos,Sin Datos,912669065,Plantea tus dudas y otros usuarios podran resp...,[],https://www.emagister.com/assets/es/logos/cent...,['l.jpg'],https://www.emagister.com/master-asesoramiento...
996,Máster universitario en orientación y mediació...,Universidad Pontificia de Salamanca,4500.0,Master oficial,Salamanca,Flexible,Sin Datos,923926833,Plantea tus dudas y otros usuarios podran resp...,[Plan de estudios:],https://www.emagister.com/assets/es/logos/cent...,['l.jpg'],https://www.emagister.com/master-universitario...
997,Curso de Mediación. Experto Universitario en M...,Instituto Español de Formación Social,1090.0,Experto universitario,Online,750h,Sí,922937155,Plantea tus dudas y otros usuarios podran resp...,"[MÓDULO I. CONFLICTO. , MÓDULO II. MEDIACIÓN. ...",https://www.emagister.com/assets/es/logos/cent...,['l.png'],https://www.emagister.com/curso-mediacion-expe...
998,Acoso y Bullying: Postgrado Experto en Prevenc...,ESNECA BUSINESS SCHOOL,380.0,Postgrado,A distancia,400h,Sin Datos,973975855,Documentos,[UNIDAD DIDÁCTICA 1. ACOSO ESCOLAR. DEFINICIÓN...,https://www.emagister.com/assets/es/logos/cent...,['l.jpg'],https://www.emagister.com/acoso-bullying-postg...


In [ ]:
DfResult.columns

In [20]:
# 03.04.- Generación Archivo parcial de resultados

# Generación Nombres Archivos a generar
from datetime import datetime

nombre=datetime.today().strftime('%d-%m-%y')

FILE_02 = "RESULT-"+nombre+".xlsx"

# Directorio donde se generara salida
path='RESULTADOS'
file=GenDirecFile(path, FILE_02)

# Escritura
DfResult.to_excel(file)
logger.info('Generado Archivo %s', FILE_02)

2022-04-02 19:44:26,662 - UOCPRA01 - INFO - Generado Archivo RESULT-02-04-22.xlsx


In [21]:
# Extracción Imagenes de las WEBs

# Construccion diccionario imagenes de las entidades

# Filtrado Entidades con Logo
#DfResultLOGO=DfResult[DfResult['URLIMAGEN']!='']

# Lista entidades unicas
listaOrganizaciones = DfResult['ENTIDAD'].unique().tolist()

# Construccion diccionario imagenes de las entidades
DiccioImagenes = {}
# Para cada entidad
for s in listaOrganizaciones:
    # Hacer entrada por entidad con lista de imagenes de la entidad
    DiccioImagenes[s] = DfResult[DfResult['ENTIDAD'] == s]['URLIMAGEN'].unique().tolist()
    
logger.info('Generado Diccionario de tamaño %s', len(DiccioImagenes))   



2022-04-02 19:46:04,415 - UOCPRA01 - INFO - Generado Diccionario de tamaño 195


In [22]:
DiccioImagenes['Instituto Español de Formación Social']

['https://www.emagister.com/assets/es/logos/centro/id/192475/size/l.png']

In [29]:
# Directorio Imagenes
Path = 'IMAGENES'

logger.info('Inicio Descarga Imagenes')

# Itera en el diccionario Previo
for k, v in DiccioImagenes.items():
    # Descarga Archivo en Path
    # v --> Lista de Links de imagenes
    # k --> Institucion
    # Archivo resultante con nombre
    logger.info('Descargando Logo de %s', k)
    # Pasar lista a String
    v = v[0]
    logger.info('Link %s', v)
    # - Eliminamos caracteres especiales
    k=re.sub(r"[^a-zA-Z0-9]"," ",k)
    # - Elimina espacios blanco principio y final
    k=k.strip()
    DescargaImagenes(Path, v, k)

logger.info('Fin Descarga Imagenes')

2022-04-02 20:04:01,076 - UOCPRA01 - INFO - Inicio Descarga Imagenes
2022-04-02 20:04:01,077 - UOCPRA01 - INFO - Descargando Logo de SELECT BUSINESS SCHOOL
2022-04-02 20:04:01,079 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/192308/size/l.png
2022-04-02 20:04:02,295 - UOCPRA01 - INFO - Descargando Logo de CESIF Formación
2022-04-02 20:04:02,297 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/56155/size/l.jpg
2022-04-02 20:04:03,580 - UOCPRA01 - INFO - Descargando Logo de Universidad Internacional de Valencia (VIU)
2022-04-02 20:04:03,582 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/124268/size/l.jpg
2022-04-02 20:04:04,884 - UOCPRA01 - INFO - Descargando Logo de ISEB - Instituto Superior Europeo de Barcelona
2022-04-02 20:04:04,885 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/1138798/size/l.jpg
2022-04-02 20:04:06,124 - UOCPRA01 - INFO - Descargando Logo de Cef -

2022-04-02 20:05:17,128 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/1145828/size/l.jpg
2022-04-02 20:05:18,399 - UOCPRA01 - INFO - Descargando Logo de ISEP, Instituto Superior de Estudios Psicológicos
2022-04-02 20:05:18,400 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/1998/size/l.jpg
2022-04-02 20:05:19,626 - UOCPRA01 - INFO - Descargando Logo de FORMAINFANCIA EUROPEAN SCHOOL
2022-04-02 20:05:19,627 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/176986/size/l.jpg
2022-04-02 20:05:20,855 - UOCPRA01 - INFO - Descargando Logo de iNeurocampus
2022-04-02 20:05:20,856 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/141298/size/l.jpg
2022-04-02 20:05:23,074 - UOCPRA01 - INFO - Descargando Logo de ELBS - ESCUELA DE LIDERAZGO
2022-04-02 20:05:23,075 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/1118013/size/l.jpg
2022-04-02 20:05:24,404 - UO

2022-04-02 20:06:28,713 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/1111907/size/l.jpg
2022-04-02 20:06:30,044 - UOCPRA01 - INFO - Descargando Logo de Universidad de Castilla La Mancha
2022-04-02 20:06:30,045 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/126892/size/l.jpg
2022-04-02 20:06:31,281 - UOCPRA01 - INFO - Descargando Logo de HOFFMANN WORLD
2022-04-02 20:06:31,282 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/1118234/size/l.png
2022-04-02 20:06:32,562 - UOCPRA01 - INFO - Descargando Logo de Andrómeda Fundación
2022-04-02 20:06:32,564 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/94042/size/l.jpg
2022-04-02 20:06:33,793 - UOCPRA01 - INFO - Descargando Logo de CEEFI International Business School
2022-04-02 20:06:33,794 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/1162731/size/l.jpg
2022-04-02 20:06:36,028 - UOCPRA01 - INFO 

2022-04-02 20:07:33,329 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/1156166/size/l.jpg
2022-04-02 20:07:34,598 - UOCPRA01 - INFO - Descargando Logo de SEAS - Estudios Superiores Abiertos
2022-04-02 20:07:34,599 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/9865/size/l.jpg
2022-04-02 20:07:35,822 - UOCPRA01 - INFO - Descargando Logo de Mediterranea Business School
2022-04-02 20:07:35,824 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/1160425/size/l.jpg
2022-04-02 20:07:37,048 - UOCPRA01 - INFO - Descargando Logo de Spain Business School - Formación
2022-04-02 20:07:37,049 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/139196/size/l.jpg
2022-04-02 20:07:39,403 - UOCPRA01 - INFO - Descargando Logo de ESCUELA DE POSTGRADO DE INGENIERIA Y ARQUITECTURA
2022-04-02 20:07:39,404 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/1155659/size/l.jpg

2022-04-02 20:08:37,919 - UOCPRA01 - INFO - Descargando Logo de Escuela de Postgrado Industrial
2022-04-02 20:08:37,919 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/1155336/size/l.jpg
2022-04-02 20:08:40,187 - UOCPRA01 - INFO - Descargando Logo de European Quality Business School
2022-04-02 20:08:40,188 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/55039/size/l.jpg
2022-04-02 20:08:41,407 - UOCPRA01 - INFO - Descargando Logo de AIDIMME - UCV
2022-04-02 20:08:41,407 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/10912/size/l.jpg
2022-04-02 20:08:43,637 - UOCPRA01 - INFO - Descargando Logo de Instituto Superior de Posgrados y Desarrollo Empresarial
2022-04-02 20:08:43,638 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/92734/size/l.jpg
2022-04-02 20:08:45,859 - UOCPRA01 - INFO - Descargando Logo de ESCUELA EUROPEA DE EXCELENCIA
2022-04-02 20:08:45,860 - UOCPRA01 - INF

2022-04-02 20:09:45,615 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/55356/size/l.jpg
2022-04-02 20:09:47,824 - UOCPRA01 - INFO - Descargando Logo de Instituto Español de Formación Social
2022-04-02 20:09:47,825 - UOCPRA01 - INFO - Link https://www.emagister.com/assets/es/logos/centro/id/192475/size/l.png
2022-04-02 20:09:49,145 - UOCPRA01 - INFO - Fin Descarga Imagenes
